In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [5]:
# take one copy for model development data
data = model_data.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [6]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [7]:
 features_list = ['Num_Enq_all_loans_last_9M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_9M',
'Num_of_PERSONAL LOAN_opened_last9M',
'num_CREDIT CARD_enq_last_6M',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'Utilization_CREDIT CARD_opened_last_36M',
'total_credit_all_open',
'num_PL_LE_1_Lac_opened_in_last_3M',
'Utilization_all',
'total_emi',
# Reconcilated Features
'percentage_ever_delinquent',
'months_since_most_recent_delinquency_installment',
#'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'month_year',
'G15_B24']

In [8]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3038214/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3038214/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 19), (13738, 19), (20606,), (13738,))

In [11]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 19) (4122, 19) (13738, 19)


In [12]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [13]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 6.0,
    'alpha': 4.9,
    'gamma': 4.4,
    'random_state': 24
}
evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 170,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.57957	eval-auc:0.56112
[10]	train-auc:0.59816	eval-auc:0.58827
[20]	train-auc:0.60007	eval-auc:0.58905
[30]	train-auc:0.60219	eval-auc:0.59624
[40]	train-auc:0.60277	eval-auc:0.59626
[50]	train-auc:0.60912	eval-auc:0.59758
[60]	train-auc:0.60919	eval-auc:0.59747
[70]	train-auc:0.62191	eval-auc:0.61877
[80]	train-auc:0.62653	eval-auc:0.61688
[90]	train-auc:0.63001	eval-auc:0.61548
[100]	train-auc:0.63028	eval-auc:0.62234
[110]	train-auc:0.63360	eval-auc:0.62275
[120]	train-auc:0.63358	eval-auc:0.61995
[130]	train-auc:0.63457	eval-auc:0.61979
[140]	train-auc:0.63471	eval-auc:0.62129
[150]	train-auc:0.63513	eval-auc:0.62058
[151]	train-auc:0.63498	eval-auc:0.62075


In [14]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.64, Gini: 0.27, KS: 0.19
Eval AUC: 0.62, Gini: 0.24, KS: 0.20
Test AUC: 0.63, Gini: 0.25, KS: 0.19
OOT AUC: 0.62, Gini: 0.24, KS: 0.18


In [15]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.142511         0.124566        0.155755        0.112788   1649   
9          0.095267         0.106533        0.112786        0.101094   1648   
8          0.066748         0.093322        0.101089        0.085959   1648   
7          0.068418         0.080431        0.085959        0.076337   1637   
6          0.068127         0.075344        0.076331        0.074660   1644   
5          0.054287         0.074245        0.074659        0.073843   1621   
4          0.051692         0.073603        0.073840        0.073335   1625   
3          0.044315         0.073040        0.073335        0.072577   1715   
2          0.032376         0.071707        0.072573        0.070909   1637   
1          0.031325         0.070466        0.070899        0.070070   1660   

        label_good  label_bad  
Decile                         
10            1414        235  
9             1491        157  
8             1538        110  
7             1525        112  
6             1532        112  
5             1533         88  
4             1541         84  
3             1639         76  
2             1584         53  
1             1608         52

In [16]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.123816         0.125773        0.155755        0.113640   1373   
9          0.082182         0.107090        0.113634        0.101532   1375   
8          0.072052         0.093818        0.101530        0.086141   1374   
7          0.067007         0.080420        0.086137        0.076378   1373   
6          0.063457         0.075361        0.076374        0.074713   1371   
5          0.052288         0.074275        0.074708        0.073882   1377   
4          0.042972         0.073616        0.073881        0.073399   1373   
3          0.044428         0.073088        0.073394        0.072783   1373   
2          0.038574         0.071859        0.072781        0.071085   1374   
1          0.026182         0.070527        0.071080        0.070070   1375   

        label_good  label_bad  
Decile                         
10            1203        170  
9             1262        113  
8             1275         99  
7             1281         92  
6             1284         87  
5             1305         72  
4             1314         59  
3             1312         61  
2             1321         53  
1             1339         36

In [17]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.135060         0.125764        0.155755        0.114068   1081   
9          0.106580         0.106717        0.114067        0.100081   1079   
8          0.092507         0.092417        0.100053        0.085384   1081   
7          0.089898         0.079869        0.085355        0.076076   1079   
6          0.080481         0.075163        0.076073        0.074599   1081   
5          0.064667         0.074197        0.074595        0.073842   1067   
4          0.060718         0.073618        0.073840        0.073414   1087   
3          0.048803         0.073136        0.073414        0.072894   1086   
2          0.047398         0.071944        0.072892        0.071055   1076   
1          0.035023         0.070516        0.071049        0.070070   1085   

        label_good  label_bad  
Decile                         
10             935        146  
9              964        115  
8              981        100  
7              982         97  
6              994         87  
5              998         69  
4             1021         66  
3             1033         53  
2             1025         51  
1             1047         38

In [18]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
0                           Num_Enq_all_loans_last_9M        79.0
14                          Num_Enq_non_BANK_last_12M        73.0
3                  Num_of_PERSONAL LOAN_opened_last9M        67.0
6                          Age_of_oldest_open_account        52.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...        28.0
9                                     Utilization_all        23.0
5                highest_sanction_amount_credit_cards        16.0
7             Utilization_CREDIT CARD_opened_last_36M        12.0
2                       num_CONSUMER LOAN_enq_last_9M        11.0
13                           No of enq in past 90days         5.0
12                       percentage_utilization_gt_75         3.0
1                      num_PERSONAL LOAN_enq_last_36M         2.0
10                         percentage_ever_delinquent         2.0
15                     Num_of_installment_loan_opened         2.0
11   month

In [19]:
feature_importance_df

Feature  Importance
0                           Num_Enq_all_loans_last_9M        79.0
14                          Num_Enq_non_BANK_last_12M        73.0
3                  Num_of_PERSONAL LOAN_opened_last9M        67.0
6                          Age_of_oldest_open_account        52.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...        28.0
9                                     Utilization_all        23.0
5                highest_sanction_amount_credit_cards        16.0
7             Utilization_CREDIT CARD_opened_last_36M        12.0
2                       num_CONSUMER LOAN_enq_last_9M        11.0
13                           No of enq in past 90days         5.0
12                       percentage_utilization_gt_75         3.0
1                      num_PERSONAL LOAN_enq_last_36M         2.0
10                         percentage_ever_delinquent         2.0
15                     Num_of_installment_loan_opened         2.0
11   months_since_most_recent_delinquency_installment         1.0
8                   num_PL_LE_1_Lac_opened_in_last_3M         1.0

In [20]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_Bureau_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_Bureau_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_Bureau_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_Bureau_Oot_Pred_1524.csv',index=False)


In [21]:
#Save the models
xgb_model.save_model('LeoPayu_Bureau_15_in_24mob.json')
print('Model Saved')

Model Saved
